In [1]:
import pandas as pd
import numpy as np
import glob
import os

from datetime import time, datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

# Data Imports

In [13]:
'''Run this cell to produce a concatenation of csvs in the directory in reverse order'''
all_files = glob.glob(os.path.join('', '*.csv'))
dtypes = {'user_type':str, 'bike_share_for_all_trip':str, 'rental_access_method':str}

data = pd.concat((pd.read_csv(f, dtype=dtypes).iloc[::-1].reset_index(drop=True) for f in all_files), ignore_index=True)

In [29]:
data.head()

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,...,June,July,August,September,October,November,Hour,Days Passed,Commute In,Commute Out
0,145,2018-01-01 00:07:41.040,2018-01-01 00:10:06.2410,316.0,San Salvador St at 1st St,37.330165,-121.885831,311.0,Paseo De San Antonio at 2nd St,37.333798,...,0,0,0,0,0,0,0,0,0,0
1,714,2018-01-01 00:07:52.943,2018-01-01 00:19:47.0750,74.0,Laguna St at Hayes St,37.776435,-122.426244,70.0,Central Ave at Fell St,37.773311,...,0,0,0,0,0,0,0,0,0,0
2,1151,2018-01-01 00:09:31.745,2018-01-01 00:28:43.1590,97.0,14th St at Mission St,37.768265,-122.420110,125.0,20th St at Bryant St,37.759200,...,0,0,0,0,0,0,0,0,0,0
3,600,2018-01-01 00:19:48.761,2018-01-01 00:29:49.0740,17.0,Embarcadero BART Station (Beale St at Market St),37.792251,-122.397086,19.0,Post St at Kearny St,37.788975,...,0,0,0,0,0,0,0,0,0,0
4,695,2018-01-01 00:19:58.761,2018-01-01 00:31:33.8320,23.0,The Embarcadero at Steuart St,37.791464,-122.391034,66.0,3rd St at Townsend St,37.778742,...,0,0,0,0,0,0,0,0,0,0


In [2]:
data = pd.read_csv('aggregated_dataset_updated.csv')
data.head()

C:\Users\Newton\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,Unnamed: 0.1,zipcode,datetime,change,rental,date,time,parking_size,num_of_muni,temp,dwpt,rhum,prcp,wdir,wspd,pres
0,0,0,94114,2018-01-31_22,-1,4,2018-01-31,22,44.012042,156,12.2,8.9,80,0.0,272.0,5.4,1018.0
1,1,1,94114,2018-01-31_23,-1,1,2018-01-31,23,44.012042,156,12.8,8.9,77,0.0,0.0,5.4,1018.3
2,2,2,94114,2018-01-31_21,2,3,2018-01-31,21,44.012042,156,12.2,8.3,77,0.0,280.0,18.4,1017.8
3,3,3,94114,2018-01-31_20,2,1,2018-01-31,20,44.012042,156,13.3,8.4,72,0.0,280.0,11.2,1017.3
4,4,4,94114,2018-01-31_19,1,7,2018-01-31,19,44.012042,156,13.9,8.3,69,0.0,280.0,13.0,1017.2


# Producing Time Series Data

Start by generating proxy columns for the date and hour to make life easier

In [3]:
data['date_hour'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d_%H')

In [4]:
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())

In [5]:
data['Business Day'] = (data['date_hour'].dt.date.isin(pd.date_range(start='2018-01-01',end='2020-01-01', freq=us_bd).date))*1

In [6]:
data['Holiday'] = (data['date_hour'].dt.date.isin(USFederalHolidayCalendar().holidays(start='2018-01-01',end='2020-01-01').date))*1

In [7]:
day_name = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
day_name = dict(enumerate(day_name))
month_name = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November']
month_name = dict(enumerate(month_name))
season_name = ['Spring', 'Summer', 'Fall']
season_name = dict(enumerate(season_name))

In [8]:
for i in range(3):
    data[season_name[i]] = (data['date_hour'].dt.month%12 // 3 + 1 == i)*1
    
for i in range(6):
    data[day_name[i]] = (data['date_hour'].dt.dayofweek == i)*1
    
for i in range(11):
    data[month_name[i]] = (data['date_hour'].dt.month == i+1)*1

In [9]:
data['Hour'] = data['date_hour'].dt.hour

In [10]:
data['Days Passed'] = (data['date_hour'] - pd.to_datetime('2018-01-01')).dt.days

In [11]:
data['Commute In'] = ((data['date_hour'].dt.time >= time(8,0)) & (data['date_hour'].dt.time <= time(9,0)) & (data['Business Day']))*1

In [12]:
data['Commute Out'] = ((data['date_hour'].dt.time >= time(16,0)) & (data['date_hour'].dt.time <= time(18,0)) & (data['Business Day']))*1

In [14]:
data.drop(columns='date_hour',inplace=True)

In [15]:
data.to_csv('aggregated_dataset_updated_with_time.csv')

# Code below for the original data

In [15]:
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())

In [16]:
data['start_time'] = pd.to_datetime(data['start_time'])

In [17]:
data['Business Day'] = (data['start_time'].dt.date.isin(pd.date_range(start='2018-01-01',end='2020-01-01', freq=us_bd).date))*1

In [18]:
data['Holiday'] = (data['start_time'].dt.date.isin(USFederalHolidayCalendar().holidays(start='2018-01-01',end='2020-01-01').date))*1

In [19]:
day_name = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
day_name = dict(enumerate(day_name))
month_name = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November']
month_name = dict(enumerate(month_name))
season_name = ['Spring', 'Summer', 'Fall']
season_name = dict(enumerate(season_name))

In [20]:
for i in range(3):
    data[season_name[i]] = (data['start_time'].dt.month%12 // 3 + 1 == i)*1

In [21]:
for i in range(6):
    data[day_name[i]] = (data['start_time'].dt.dayofweek == i)*1

In [22]:
for i in range(11):
    data[month_name[i]] = (data['start_time'].dt.month == i+1)*1

In [23]:
data['Hour'] = data['start_time'].dt.hour

In [24]:
data['Days Passed'] = (data['start_time'] - pd.to_datetime('2018-01-01')).dt.days

In [25]:
data['Commute In'] = ((data['start_time'].dt.time >= time(8,0)) & (data['start_time'].dt.time <= time(9,30)) & (data['Business Day']))*1

In [26]:
data['Commute Out'] = ((data['start_time'].dt.time >= time(16,30)) & (data['start_time'].dt.time <= time(18,30)) & (data['Business Day']))*1

In [27]:
export_df = data.loc[:, 'Business Day':]

In [28]:
export_df.to_csv('time_series_nonagg.csv')

In [ ]:
# autoregressive features are most easily added after we groupby and/or reindex by time and location, so we can use the shift() method to shift per hour
# data['Lag'] = data['Change in Bikes'].shift()